In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\Lung-Infection\\notebooks'

In [2]:
import os 


os.chdir('../')
%pwd

'c:\\Users\\44787\\Desktop\\Lung-Infection'

In [3]:
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib


import warnings
warnings.filterwarnings("ignore")

In [26]:
mlflow.set_tracking_uri('http://localhost:2024')

runs = mlflow.search_runs(experiment_ids=[1])

runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.validation_loss,metrics.val_accuracy,metrics.Evaluation Accuracy,metrics.val_loss,...,params.validation_freq,params.opt_weight_decay,params.opt_gradient_accumulation_steps,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.latest_checkpoint_artifact,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.log-model.history
0,17f48c61db08418db3f78bf59100fc5f,1,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-12 20:00:21.367000+00:00,2024-06-12 20:08:52.788000+00:00,0.490341,0.755,0.755,0.490341,...,1,None,None,.\src\pipeline\stage_02_training_and_evaluatio...,29310121b2be6cb9beafe2e6222005efd828ab6e,LOCAL,checkpoints/latest_checkpoint.h5,44787,Lung Infection Experiments,"[{""run_id"": ""17f48c61db08418db3f78bf59100fc5f""..."
1,1d9f4bb3d0fc48f8b416bf70118bbf7c,1,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-12 19:52:56.246000+00:00,2024-06-12 19:56:30.377000+00:00,0.492981,0.785,0.785,0.492981,...,1,None,None,.\src\pipeline\stage_02_training_and_evaluatio...,29310121b2be6cb9beafe2e6222005efd828ab6e,LOCAL,checkpoints/latest_checkpoint.h5,44787,Lung Infection Experiments,"[{""run_id"": ""1d9f4bb3d0fc48f8b416bf70118bbf7c""..."
2,9df359a63b06434d87d294649c2da024,1,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-12 19:44:27.654000+00:00,2024-06-12 19:46:51.940000+00:00,0.476958,0.775,0.775,0.476958,...,1,None,None,.\src\pipeline\stage_02_training_and_evaluatio...,29310121b2be6cb9beafe2e6222005efd828ab6e,LOCAL,checkpoints/latest_checkpoint.h5,44787,Lung Infection Experiments,"[{""run_id"": ""9df359a63b06434d87d294649c2da024""..."


In [27]:
# delete the run
# mlflow.delete_run('2c185a00f9a74df3bee7bb203d143e34')

In [28]:
runs['metrics.validation_accuracy'].sort_values(ascending=False, ignore_index=True)

0    0.785
1    0.775
2    0.755
Name: metrics.validation_accuracy, dtype: float64

In [29]:
highest = runs["metrics.validation_accuracy"].sort_values(ascending=False, ignore_index=True)[0]
highest

0.7850000262260437

In [30]:
runs[runs["metrics.validation_accuracy"]==highest]

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.validation_loss,metrics.val_accuracy,metrics.Evaluation Accuracy,metrics.val_loss,...,params.validation_freq,params.opt_weight_decay,params.opt_gradient_accumulation_steps,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.latest_checkpoint_artifact,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.log-model.history
1,1d9f4bb3d0fc48f8b416bf70118bbf7c,1,FINISHED,file:///C:/Users/44787/Desktop/Lung-Infection/...,2024-06-12 19:52:56.246000+00:00,2024-06-12 19:56:30.377000+00:00,0.492981,0.785,0.785,0.492981,...,1,None,None,.\src\pipeline\stage_02_training_and_evaluatio...,29310121b2be6cb9beafe2e6222005efd828ab6e,LOCAL,checkpoints/latest_checkpoint.h5,44787,Lung Infection Experiments,"[{""run_id"": ""1d9f4bb3d0fc48f8b416bf70118bbf7c""..."


In [31]:
runs[runs["metrics.validation_accuracy"]==highest]['run_id']

1    1d9f4bb3d0fc48f8b416bf70118bbf7c
Name: run_id, dtype: object

In [32]:
highest_run_id = runs[runs["metrics.validation_accuracy"]==highest]['run_id']
highest_run_id = highest_run_id.reset_index(drop=True)[0]
highest_run_id

'1d9f4bb3d0fc48f8b416bf70118bbf7c'

In [33]:
model_name = 'EfficientNetV2M'

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)
    # print(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)

{   'aliases': [],
    'creation_timestamp': 1718222190359,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1718222190359,
    'name': 'EfficientNetV2M',
    'run_id': '1d9f4bb3d0fc48f8b416bf70118bbf7c',
    'run_link': '',
    'source': 'file:///C:/Users/44787/Desktop/Lung-Infection/mlflow_artifacts/1/1d9f4bb3d0fc48f8b416bf70118bbf7c/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


In [34]:
model_name = 'EfficientNetV2M'

client = MlflowClient()
for mv in client.search_model_versions(f"name='{model_name}'"):
    mv = dict(mv)

    if mv["run_id"] == highest_run_id:
        current_version = mv["version"]
        logged_model = mv["source"]
        pprint(mv, indent=4)
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Production"
        )
    else:
        current_version = mv["version"]
        client.transition_model_version_stage(
            name=model_name,
            version=current_version,
            stage="Staging"
        )  

{   'aliases': [],
    'creation_timestamp': 1718222190359,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1718222190359,
    'name': 'EfficientNetV2M',
    'run_id': '1d9f4bb3d0fc48f8b416bf70118bbf7c',
    'run_link': '',
    'source': 'file:///C:/Users/44787/Desktop/Lung-Infection/mlflow_artifacts/1/1d9f4bb3d0fc48f8b416bf70118bbf7c/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}


In [37]:
loaded_model = mlflow.pyfunc.load_model(logged_model)

model_path = os.path.join('artifacts\ModelTrainings\mlflow_model.h5')
joblib.dump(loaded_model, model_path)

['artifacts\\ModelTrainings\\mlflow_model.h5']